In [1]:
import genanki
from lib import processor

model = genanki.Model(
  1647878250,
  'Stiefo Model',
  fields=[
    {'name': 'Picture'},
    {'name': 'Text'},
  ],
  templates=[
    {
      'name': 'Text -> Picture',
      'qfmt': '{{Text}}',
      'afmt': '{{FrontSide}}<hr id="answer">{{Picture}}',
    },
    {
      'name': 'Picture -> Text',
      'qfmt': '{{Picture}}',
      'afmt': '{{FrontSide}}<hr id="answer">{{Text}}',
    },
    ],
    css=".card {font-family: arial;font-size: 20px;text-align: center;color: black;background-color: white;}"
  )

stiefo_deck = genanki.Deck(542127561267, "Stiefographie")


In [2]:
class StiefoNote(genanki.Note):
    @property
    def guid(self):
        return genanki.guid_for(self.fields[0])

In [3]:
dek_filename = "dek_anki_deck.csv"
import csv
from typing import List
from tqdm import tqdm

media_files: List[str] = []

forbidden_chars = ["(", ")", "-", "v2", "é", "~", "'", ":", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", ",", "î", "’"]

with open(dek_filename, "r", encoding="utf-8") as f:

    line_count = sum(1 for _ in f)

    f.seek(0)

    for entry in tqdm(csv.reader(f, delimiter=","), total=line_count):
        word, tag = entry
        word = word.replace("_", " ").replace(".","").replace("!", "").replace("?", "")

        if any([char in word for char in forbidden_chars]):
            continue
    
        svg: str = processor.text_to_svg(word)

        note = StiefoNote(
            model=model,
            fields=[f'{svg}', word],
            tags=[tag.replace(" ", "")]
        )
        stiefo_deck.add_note(note)


100%|██████████| 38116/38116 [00:06<00:00, 5685.50it/s]


In [4]:
export = genanki.Package(stiefo_deck)

export.write_to_file("stiefo_deck.apkg")